# Read real and generated datasets 

In [4]:
import scanpy as sc
from celldreamer.eval.compute_evaluation_metrics import process_labels, compute_evaluation_metrics
from scipy import sparse
import pandas as pd

AttributeError: module 'scipy.sparse' has no attribute 'linalg'

In [ ]:
def add_to_dict(d, metrics):
    for metric in metrics:
        if metric not in d:
            d[metric] = [metrics[metric]]
        else:
            d[metric]+=[metrics[metric]]
    return d

In [ ]:
results_celldreamer = {}
results_scvi = {} 
results_scdiff = {}

for i in range(3):
    adata_generated_path_celldreamer = f"/home/icb/alessandro.palma/environment/celldreamer/project_folder/datasets/generated/dentategyrus/generated_cells_{i}.h5ad"
    adata_generated_celldreamer = sc.read_h5ad(adata_generated_path_celldreamer)
    sc.pp.subsample(adata_generated_celldreamer, n_obs=10000)

    adata_generated_path_scvi = f"/home/icb/alessandro.palma/environment/celldreamer/project_folder/baseline_experiments/scvi/dentategyrus/generated/dentategyrus_{i}.h5ad"
    adata_generated_scvi = sc.read_h5ad(adata_generated_path_scvi)
    sc.pp.subsample(adata_generated_scvi, n_obs=10000)
    adata_generated_scvi.X = sparse.csr_matrix(adata_generated_scvi.X)
    adata_generated_scvi = process_labels(adata_real, adata_generated_scvi, "clusters", categorical_obs=True)

    adata_generated_path_scDiffusion = f"/home/icb/alessandro.palma/environment/celldreamer/project_folder/baseline_experiments/scDiffusion/generated/dentategyrus/generated_cells_{i}.h5ad"
    adata_generated_scDiffusion = sc.read_h5ad(adata_generated_path_scDiffusion)
    sc.pp.subsample(adata_generated_scDiffusion, n_obs=10000)
    adata_generated_scDiffusion.X = sparse.csr_matrix(adata_generated_scDiffusion.X)
    adata_generated_scDiffusion = process_labels(adata_real, adata_generated_scDiffusion, "clusters", categorical_obs=False)

    results_celldreamer_i = compute_evaluation_metrics(adata_real, adata_generated_celldreamer, "clusters", conditional=True, nn=10)
    results_scvi_i = compute_evaluation_metrics(adata_real, adata_generated_scvi, "clusters", conditional=True, nn=10)
    results_scdiff_i = compute_evaluation_metrics(adata_real, adata_generated_scDiffusion, "clusters", conditional=True, nn=10)

    results_celldreamer = add_to_dict(results_celldreamer, results_celldreamer_i)
    results_scvi = add_to_dict(results_scvi, results_scvi_i)
    results_scdiff = add_to_dict(results_scdiff, results_scdiff_i)